# Predicting Graduation Rates - Regression and Classification 

## What's in this Notebook

Regression and Classification models. I tried a variety of models and found pretty similar results in terms of:

* inability to predict graduation rates very well, and 
* similar features identified as most important for predicting (such as it is) graduation rates

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV

In [6]:
grad = pd.read_csv('./grad2.csv')

## Regression and Classifiction Models
I chose features that correlated .10 or greater with the ALL_RATE_1112 variable.  I tried variables models, lineary with regularlization, decision tree regression, bagging, and random forest, all with pretty similar results. R-squared never got much above .20, but the top features definitely showed some similarity in terms of the features that were most important. 


In [201]:
#acs variables
y = grad['ALL_RATE_1112']
features = ['Med_HHD_Inc_ACS_08_12', 'pct_Tot_Occp_Units_ACS_08_12', 'pct_Tot_Occp_Units_CEN_2010',
            'pct_College_ACS_08_12','pct_MrdCple_HHD_CEN_2010','pct_MrdCple_HHD_ACS_08_12',
            'pct_URBANIZED_AREA_POP_CEN_2010','pct_Civ_emp_16p_ACS_08_12','pct_Civ_emp_25_44_ACS_08_12',
            'pct_Rel_Family_HHD_ACS_08_12','pct_Rel_Family_HHDS_CEN_2010','pct_Owner_Occp_HU_CEN_2010',
            'pct_Owner_Occp_HU_ACS_08_12','pct_Civ_emp_16_24_ACS_08_12','pct_US_Cit_Nat_ACS_08_12',
            'pct_Civ_unemp_16_24_ACS_08_12','pct_Renter_Occp_HU_ACS_08_12','pct_Renter_Occp_HU_CEN_2010',
            'pct_NO_PH_SRVC_ACS_08_12','pct_NonFamily_HHD_CEN_2010','pct_NonFamily_HHD_ACS_08_12',
            'pct_Sngl_Prns_HHD_ACS_08_12','pct_Sngl_Prns_HHD_CEN_2010','pct_Civ_unemp_25_44_ACS_08_12',
            'pct_Civ_unemp_16p_ACS_08_12','pct_Female_No_HB_ACS_08_12','pct_Crowd_Occp_U_ACS_08_12',
            'pct_Female_No_HB_CEN_2010','pct_PUB_ASST_INC_ACS_08_12','pct_Mobile_Homes_ACS_08_12',
            'pct_Not_MrdCple_HHD_ACS_08_12','pct_RURAL_POP_CEN_2010','pct_Not_HS_Grad_ACS_08_12',
            'pct_Not_MrdCple_HHD_CEN_2010','pct_Vacant_Units_CEN_2010','pct_Prs_Blw_Pov_Lev_ACS_08_12', 
            'pct_Vacant_Units_ACS_08_12','pct_No_Plumb_ACS_08_12']
X = features_to_use[features]

In [202]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .4, random_state = 42)

In [203]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

### Linear Regression
Initial cross-validation score on the ACS variables is pretty low, .18. 

The r-squared values were very low, in the .17 to .21 range. I tried Lasso and Ridge regularlization on the ACS variables, with no improvement. 

The RMSE was 10.8, so on average I would be off 10.8% in predicting graduation rates, which is huge. 

I decided to go forward with other types of analyses, just assuming that there are so many other factors that go into whether or not someone will graduate from high school that are not captured by census data. 

In [204]:
lr = LinearRegression()
cross_val_score(lr, X_train_scaled, y_train).mean()

0.18198111616394871

In [205]:
lr = linear_model.LinearRegression()
lr.fit(X_train_scaled, y_train)
lr.score(X_train_scaled, y_train)

0.20428609677296805

In [206]:
lr.fit(X_test_scaled, y_test)
lr.score(X_test_scaled, y_test)

0.1887447354486944

Ridge regularization on the ACS variables - Did not improve the R2

In [207]:
r_alphas = np.logspace(0, 5, 200) #creates 200 alphas
coefs = [] #want to save all coefs when running the models
for alpha in r_alphas:
    model = Ridge(alpha = alpha)
    model.fit(X_test_scaled, y_test)
    coefs.append(model.coef_)

In [208]:
best_ridge = RidgeCV(alphas = r_alphas)
best_ridge.fit(X_test_scaled, y_test)
best_ridge.score(X_test_scaled, y_test)

0.18442727322089136

Lasso regularlization on the ACS variables - also did not improve the R2

In [209]:
l_alphas = np.arange(.001, .15, .0025) #starts at .001, skips .0025, ends at .15
coefs = []
for alpha in l_alphas:
    model = Lasso(alpha = alpha)
    model.fit(X_test_scaled, y_test)
    coefs.append(model.coef_)

/Users/Morgan2/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [210]:
best_lasso = LassoCV(alphas = l_alphas)
best_lasso.fit(X_test_scaled, y_test)
best_lasso.score(X_test_scaled, y_test)

0.18553771987141032

In [211]:
# Coefficients from the ACS variables
coef_df = pd.DataFrame(lr.coef_, columns = ['coef'])
coef_df['features'] = features
coef_df.sort_values('coef', ascending = False)

,coef,features
13,5.958058e+08,pct_Civ_emp_16_24_ACS_08_12
15,5.958058e+08,pct_Civ_unemp_16_24_ACS_08_12
24,4.560099e+07,pct_Civ_unemp_16p_ACS_08_12
7,4.560099e+07,pct_Civ_emp_16p_ACS_08_12
19,4.465945e+03,pct_NonFamily_HHD_CEN_2010
10,4.460650e+03,pct_Rel_Family_HHDS_CEN_2010
27,1.319462e+00,pct_Female_No_HB_CEN_2010
32,5.645553e-01,pct_Not_HS_Grad_ACS_08_12
25,5.547102e-01,pct_Female_No_HB_ACS_08_12
18,4.801840e-01,pct_NO_PH_SRVC_ACS_08_12


In [212]:
predictions  =  model.predict(X_test_scaled)
predictions

array([83.33889259, 86.52761969, 88.0706558 , ..., 85.25863462,
       83.79911158, 86.04868686])

The root mean squared error tells me that on average I am off by 10% in predicting graduation rates, which is pretty dang terrible. 

In [213]:
from sklearn.metrics import mean_squared_error

np.sqrt(mean_squared_error(y_test, predictions))

10.828608054448994

### Decision Tree Regression Modeling

In [214]:
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

dtr_params = {
    'max_depth':[None,1,2,3,4],
    'max_features':[None,'log2','sqrt',2,3,4,5],
    'min_samples_split':[2,3,4,5,10,15,20,25,30,40,50]
}

dtr_gs = GridSearchCV(DecisionTreeRegressor(), dtr_params, cv=5, verbose=1)

In [215]:
dtr_gs.fit(X_train, y_train)
print(dtr_gs.best_params_)
print(dtr_gs.best_score_)

Fitting 5 folds for each of 385 candidates, totalling 1925 fits
{'max_depth': 4, 'max_features': 5, 'min_samples_split': 5}
0.15849667836740677


[Parallel(n_jobs=1)]: Done 1925 out of 1925 | elapsed:   43.6s finished


In [216]:
dtr_gs.score(X_test, y_test)

0.13071473804439238

These are the features that came out as most important by the Decision Tree Regressor.

In [217]:
dtr_best = dtr_gs.best_estimator_
best_features = dtr_best.feature_importances_
pd.DataFrame(best_features)
fi = pd.DataFrame({
        'feature':X.columns,
        'importance':dtr_best.feature_importances_
    })

fi.sort_values('importance', ascending=False, inplace=True)
fi

,feature,importance
0,Med_HHD_Inc_ACS_08_12,0.378640
34,pct_Vacant_Units_CEN_2010,0.207369
27,pct_Female_No_HB_CEN_2010,0.147216
3,pct_College_ACS_08_12,0.100970
28,pct_PUB_ASST_INC_ACS_08_12,0.027867
31,pct_RURAL_POP_CEN_2010,0.027150
4,pct_MrdCple_HHD_CEN_2010,0.026174
2,pct_Tot_Occp_Units_CEN_2010,0.020818
22,pct_Sngl_Prns_HHD_CEN_2010,0.015224
24,pct_Civ_unemp_16p_ACS_08_12,0.013854


In [218]:
y = grad['ALL_RATE_1112']
important_features = ['pct_Tot_Occp_Units_CEN_2010',
'pct_Female_No_HB_CEN_2010',
'Med_HHD_Inc_ACS_08_12',
'pct_PUB_ASST_INC_ACS_08_12',
'pct_No_Plumb_ACS_08_12',
'pct_College_ACS_08_12',
'pct_Crowd_Occp_U_ACS_08_12',
'pct_Civ_emp_16p_ACS_08_12']
X2 = features_to_use[important_features]

### Ensemble Models

### Bagging
I only remembered after I did this model that I cannot look at feature importances with a Bagging model. 

In [219]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, RandomForestClassifier

bag = BaggingRegressor(random_state=42)
bag_params = {
#    'base_estimator': [None, LogisticRegression()], #default is decision tree, can include others
#    'base_estimator__penalty': ['l1', 'l2']
#    'base_estimator__criterion' ['gini', 'entropy'] # use this if using DecisionTreeClassifier
    'n_estimators': [5,10, 15, 20, 25], #number of trees
    'max_samples': [.5, 1.0], #.5 means each tree gets half the rows
    
}
gs_bag = GridSearchCV(bag, bag_params)
gs_bag.fit(X_train, y_train)
print(gs_bag.best_score_)
gs_bag.best_params_

0.19299298855441277


{'max_samples': 0.5, 'n_estimators': 25}

In [220]:
gs_bag.score(X_test, y_test)

0.24416352228059712

### Random Forest Regressor

In [221]:
rf = RandomForestRegressor(random_state = 42)
rf_params ={
    'n_estimators': [5, 10, 15, 20, 25],
    'max_depth': [None, 1, 2, 3],
    'max_features': ['auto', 'log2'],
}
gs_rf = GridSearchCV(rf, rf_params)
gs_rf.fit(X_train, y_train)
print(gs_rf.best_score_)
print(gs_rf.best_params_)

0.1992321445661966
{'max_depth': None, 'max_features': 'log2', 'n_estimators': 25}


In [222]:
gs_rf.score(X_test, y_test)

0.23669522553306

These are the feature importances from the random forest model. Interesting how this model assigns importance values to many more features than the decision tree model above. I guess this is because the random forest model uses different combination of features, so ends up keeping more of them. 

In [223]:
rf_best = gs_rf.best_estimator_
best_features = rf_best.feature_importances_
pd.DataFrame(best_features)

fi = pd.DataFrame({
        'feature':X.columns,
        'importance':rf_best.feature_importances_
    })

fi.sort_values('importance', ascending=False, inplace=True)
fi

,feature,importance
27,pct_Female_No_HB_CEN_2010,0.052376
36,pct_Vacant_Units_ACS_08_12,0.047912
1,pct_Tot_Occp_Units_ACS_08_12,0.045944
2,pct_Tot_Occp_Units_CEN_2010,0.041215
32,pct_Not_HS_Grad_ACS_08_12,0.036655
35,pct_Prs_Blw_Pov_Lev_ACS_08_12,0.036091
37,pct_No_Plumb_ACS_08_12,0.034098
34,pct_Vacant_Units_CEN_2010,0.033056
3,pct_College_ACS_08_12,0.030686
24,pct_Civ_unemp_16p_ACS_08_12,0.029888


### Classification Models
For these, I created a target variable, grad_categories, where grad rates above the mean were classified as 1, and rates below the mean were classified as 0. The value counts show a fairly balanced split between 0's and 1's.

The average graduation rate is 83%, and all the classification models I ran failed to get even close to that level. The best accuracy score was only in the mid 60%. 

In [224]:
grad['grad_categories'].value_counts()

1    5564
0    4031
Name: grad_categories, dtype: int64

In [225]:
grad['ALL_RATE_1112'].mean()

83.06748306409588

In [226]:
y = grad['grad_categories']
features = ['Med_HHD_Inc_ACS_08_12', 'pct_Tot_Occp_Units_ACS_08_12', 'pct_Tot_Occp_Units_CEN_2010',
            'pct_College_ACS_08_12','pct_MrdCple_HHD_CEN_2010','pct_MrdCple_HHD_ACS_08_12',
            'pct_URBANIZED_AREA_POP_CEN_2010','pct_Civ_emp_16p_ACS_08_12','pct_Civ_emp_25_44_ACS_08_12',
            'pct_Rel_Family_HHD_ACS_08_12','pct_Rel_Family_HHDS_CEN_2010','pct_Owner_Occp_HU_CEN_2010',
            'pct_Owner_Occp_HU_ACS_08_12','pct_Civ_emp_16_24_ACS_08_12','pct_US_Cit_Nat_ACS_08_12',
            'pct_Civ_unemp_16_24_ACS_08_12','pct_Renter_Occp_HU_ACS_08_12','pct_Renter_Occp_HU_CEN_2010',
            'pct_NO_PH_SRVC_ACS_08_12','pct_NonFamily_HHD_CEN_2010','pct_NonFamily_HHD_ACS_08_12',
            'pct_Sngl_Prns_HHD_ACS_08_12','pct_Sngl_Prns_HHD_CEN_2010','pct_Civ_unemp_25_44_ACS_08_12',
            'pct_Civ_unemp_16p_ACS_08_12','pct_Female_No_HB_ACS_08_12','pct_Crowd_Occp_U_ACS_08_12',
            'pct_Female_No_HB_CEN_2010','pct_PUB_ASST_INC_ACS_08_12','pct_Mobile_Homes_ACS_08_12',
            'pct_Not_MrdCple_HHD_ACS_08_12','pct_RURAL_POP_CEN_2010','pct_Not_HS_Grad_ACS_08_12',
            'pct_Not_MrdCple_HHD_CEN_2010','pct_Vacant_Units_CEN_2010','pct_Prs_Blw_Pov_Lev_ACS_08_12', 
            'pct_Vacant_Units_ACS_08_12','pct_No_Plumb_ACS_08_12']
X = features_to_use[features]

In [227]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .4, random_state = 42)

In [228]:
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

### Logistic Regression

In [229]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix
logr = LogisticRegression()

In [230]:
logr.fit(X_train_scaled, y_train)
train_model = logr.score(X_train_scaled, y_train)
train_model

0.6588500955358694

In [231]:
test_model_score = logr.score(X_test_scaled,y_test)
test_model_score

0.6490359562272017

In [232]:
predictions = logr.predict(X_test_scaled)

I think this confusion matrix is saying that the model does better at correctly predicting true low graduations (1743) than true high graduation rates (748). 

In [233]:
cm = confusion_matrix(y_test, predictions)
cm

array([[ 748,  857],
       [ 490, 1743]])

### Decision Tree Classification

In [234]:
dtr_params = {
    'max_depth':[None,1,2,3,4],
    'max_features':[None,'log2','sqrt',2,3,4,5],
    'min_samples_split':[2,3,4,5,10,15,20,25,30,40,50]
}

dtc_gs = GridSearchCV(DecisionTreeClassifier(), dtr_params, cv=5, verbose=1)

In [235]:
dtc_gs.fit(X_train, y_train)
print(dtc_gs.best_params_)
print(dtc_gs.best_score_)

Fitting 5 folds for each of 385 candidates, totalling 1925 fits
{'max_depth': 3, 'max_features': 'sqrt', 'min_samples_split': 30}
0.6571130797290256


[Parallel(n_jobs=1)]: Done 1925 out of 1925 | elapsed:   45.4s finished


In [236]:
dtc_gs.score(X_test, y_test)

0.6474726420010423

In [237]:
dtc_best = dtc_gs.best_estimator_
best_features = dtc_best.feature_importances_
pd.DataFrame(best_features)

fi = pd.DataFrame({
        'feature':X.columns,
        'importance':dtc_best.feature_importances_
    })

fi.sort_values('importance', ascending=False, inplace=True)
fi

,feature,importance
1,pct_Tot_Occp_Units_ACS_08_12,0.439413
5,pct_MrdCple_HHD_ACS_08_12,0.156529
34,pct_Vacant_Units_CEN_2010,0.138270
33,pct_Not_MrdCple_HHD_CEN_2010,0.090548
7,pct_Civ_emp_16p_ACS_08_12,0.083948
32,pct_Not_HS_Grad_ACS_08_12,0.054601
37,pct_No_Plumb_ACS_08_12,0.036691
35,pct_Prs_Blw_Pov_Lev_ACS_08_12,0.000000
36,pct_Vacant_Units_ACS_08_12,0.000000
31,pct_RURAL_POP_CEN_2010,0.000000


### Random Forest Classification

In [238]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, RandomForestClassifier
rf = RandomForestClassifier(random_state = 42)
rf_params ={
    'n_estimators': [5, 10, 15, 20, 25],
    'max_depth': [None, 1, 2, 3],
    'max_features': ['auto', 'log2'],
}
gs_rf = GridSearchCV(rf, rf_params)
gs_rf.fit(X_train, y_train)
print(gs_rf.best_score_)
print(gs_rf.best_params_)

0.6746569393781483
{'max_depth': None, 'max_features': 'auto', 'n_estimators': 25}


In [239]:
gs_rf.score(X_test, y_test)

0.6685773840541949

In [240]:
rf_best = gs_rf.best_estimator_
best_features = rf_best.feature_importances_
pd.DataFrame(best_features)

fi = pd.DataFrame({
        'feature':X.columns,
        'importance':rf_best.feature_importances_
    })

fi.sort_values('importance', ascending=False, inplace=True)
fi

,feature,importance
27,pct_Female_No_HB_CEN_2010,0.052891
36,pct_Vacant_Units_ACS_08_12,0.040580
35,pct_Prs_Blw_Pov_Lev_ACS_08_12,0.037309
0,Med_HHD_Inc_ACS_08_12,0.037240
32,pct_Not_HS_Grad_ACS_08_12,0.035969
1,pct_Tot_Occp_Units_ACS_08_12,0.035036
2,pct_Tot_Occp_Units_CEN_2010,0.034149
3,pct_College_ACS_08_12,0.033255
33,pct_Not_MrdCple_HHD_CEN_2010,0.028976
24,pct_Civ_unemp_16p_ACS_08_12,0.028825
